Training on GPU: NVIDIA GeForce RTX 2080 SUPER

--> For training cfg/yolov4-custom.cfg download the pre-trained weights-file (162 MB): yolov4.conv.137 (Google      drive mirror yolov4.conv.137 )
    ??? Why not the original weight file which was used for inferencing ???
    
--> Create file yolo-obj.cfg with the same content as in yolov4-custom.cfg:
    * change line batch to batch=64
    * change line subdivisions to subdivisions=16
    * change line max_batches to (classes*2000 but not less than number of training images, but not less than number of training images and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
    * change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
    * set network size width=416 height=416 or any value multiple of 32:
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L8-L9
    * change line classes=80 to your number of objects in each of 3 [yolo]-layers:
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L610
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L696
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L783
    * change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L603
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L689
      - https://github.com/AlexeyAB/darknet/blob/0039fd26786ab5f71d5af725fc18b3f521e7acfd/cfg/yolov3.cfg#L776
    * when using [Gaussian_yolo] layers, change [filters=57] filters=(classes + 9)x3 in the 3 [convolutional] before each [Gaussian_yolo] layer
      - https://github.com/AlexeyAB/darknet/blob/6e5bdf1282ad6b06ed0e962c3f5be67cf63d96dc/cfg/Gaussian_yolov3_BDD.cfg#L604
      - https://github.com/AlexeyAB/darknet/blob/6e5bdf1282ad6b06ed0e962c3f5be67cf63d96dc/cfg/Gaussian_yolov3_BDD.cfg#L696
      - https://github.com/AlexeyAB/darknet/blob/6e5bdf1282ad6b06ed0e962c3f5be67cf63d96dc/cfg/Gaussian_yolov3_BDD.cfg#L789
      - This is probably not applicable
    * So if classes=1 then should be filters=18. If classes=2 then write filters=21
        (Do not write in the cfg-file: filters=(classes + 5)x3)
        (Generally filters depends on the classes, coords and number of masks, i.e. filters=(classes + coords + 1)*<number of mask>, where mask is indices of anchors. If mask is absence, then filters=(classes + coords + 1)*num)
        So for example, for 2 objects, your file yolo-obj.cfg should differ from yolov4-custom.cfg in such lines in each of 3 [yolo]-layers:

            [convolutional]
            filters=21

            [region]
            classes=2
            
--> Create file obj.names in the directory build\darknet\x64\data\, with objects names - each in new line
--> Create file obj.data in the directory build\darknet\x64\data\, containing (where classes = number of objects):
            classes = 2
            train  = data/train.txt
            valid  = data/test.txt
            names = data/obj.names
            backup = backup/

--> Put image-files (.jpg) of your objects in the directory build\darknet\x64\data\obj\
--> You should label each object on images from your dataset.
        It will create .txt-file for each .jpg-image-file - in the same directory and with the same name, but with .txt-extension, and put to file: object number and object coordinates on this image, for each object in new line:

            <object-class> <x_center> <y_center> <width> <height>

        Where:

            <object-class> - integer object number from 0 to (classes-1)
            <x_center> <y_center> <width> <height> - float values relative to width and height of image, it can be equal from (0.0 to 1.0]
            for example: <x> = <absolute_x> / <image_width> or <height> = <absolute_height> / <image_height>
            atention: <x_center> <y_center> - are center of rectangle (are not top-left corner)

--> Create file train.txt in directory build\darknet\x64\data\, with filenames of your images, each filename in new line, with path relative to darknet.exe, for example containing:
        data/obj/img1.jpg
        data/obj/img2.jpg
        data/obj/img3.jpg
        
        
--> Download pre-trained weights for the convolutional layers and put to the directory build\darknet\x64
        for yolov4.cfg, yolov4-custom.cfg (162 MB): yolov4.conv.137 (Google drive mirror yolov4.conv.137 )
        
--> Start training by using the command line: darknet.exe detector train data/obj.data yolo-obj.cfg yolov4.conv.137
         use ./darknet instead of darknet.exe for using it in Linux or MacOS
    - file yolo-obj_last.weights will be saved to the build\darknet\x64\backup\ for each 100 iterations
    - file yolo-obj_xxxx.weights will be saved to the build\darknet\x64\backup\ for each 1000 iterations
    - to disable Loss-Window use darknet.exe detector train data/obj.data yolo-obj.cfg yolov4.conv.137 -dont_show, if you train on computer without monitor like a cloud Amazon EC2
    - to see the mAP & Loss-chart during training on remote server without GUI, use command darknet.exe detector train data/obj.data yolo-obj.cfg yolov4.conv.137 -dont_show -mjpeg_port 8090 -map then open URL http://ip-address:8090 in Chrome/Firefox browser
    
--> For training with mAP (mean average precisions) calculation for each 4 Epochs (set valid=valid.txt or train.txt in obj.data file) and run:      darknet.exe detector train data/obj.data yolo-obj.cfg yolov4.conv.137 -map
--> After training is complete - get result yolo-obj_final.weights from path build\darknet\x64\backup\
--> After each 100 iterations you can stop and later start training from this point. For example, after 2000 iterations you can stop training,      and later just start training using: darknet.exe detector train data/obj.data yolo-obj.cfg backup\yolo-obj_2000.weights
--> Also you can get result earlier than all 45000 iterations

    *    Note: If during training you see nan values for avg (loss) field - then training goes wrong, but if nan is in some other lines - then      training goes well.
    *    Note: If you changed width= or height= in your cfg-file, then new width and height must be divisible by 32.
    *   Note: After training use such command for detection: darknet.exe detector test data/obj.data yolo-obj.cfg yolo-obj_8000.weights
    *    Note: if error Out of memory occurs then in .cfg-file you should increase subdivisions=16, 32 or 64: link



How to improve object detection?
--> Before training:
    *   set flag random=1 in your .cfg-file - it will increase precision by training Yolo for different resolutions: link

    *   increase network resolution in your .cfg-file (height=608, width=608 or any value multiple of 32) - it will increase precision
    
    *   check that each object that you want to detect is mandatory labeled in your dataset - no one object in your data set should not be         without label. In the most training issues - there are wrong labels in your dataset (got labels by using some conversion script,           marked with a third-party tool, ...). Always check your dataset by using: https://github.com/AlexeyAB/Yolo_mark

    *   my Loss is very high and mAP is very low, is training wrong? Run training with -show_imgs flag at the end of training command, do you     see correct bounded boxes of objects (in windows or in files aug_...jpg)? If no - your training dataset is wrong.

    *   for each object which you want to detect - there must be at least 1 similar object in the Training dataset with about the same: shape,     side of object, relative size, angle of rotation, tilt, illumination. So desirable that your training dataset include images with         objects at diffrent: scales, rotations, lightings, from different sides, on different backgrounds - you should preferably have 2000       different images for each class or more, and you should train 2000*classes iterations or more

    *   desirable that your training dataset include images with non-labeled objects that you do not want to detect - negative samples without     bounded box (empty .txt files) - use as many images of negative samples as there are images with objects

    *   What is the best way to mark objects: label only the visible part of the object, or label the visible and overlapped part of the           object, or label a little more than the entire object (with a little gap)? Mark as you like - how would you like it to be detected.

    *   for training with a large number of objects in each image, add the parameter max=200 or higher value in the last [yolo]-layer or           [region]-layer in your cfg-file (the global maximum number of objects that can be detected by YoloV3 is 0,0615234375*(width*height)       where are width and height are parameters from [net] section in cfg-file)

    *   for training for small objects (smaller than 16x16 after the image is resized to 416x416) - set layers = 23 instead of                     https://github.com/AlexeyAB/darknet/blob/6f718c257815a984253346bba8fb7aa756c55090/cfg/yolov4.cfg#L895

        - set stride=4 instead of https://github.com/AlexeyAB/darknet/blob/6f718c257815a984253346bba8fb7aa756c55090/cfg/yolov4.cfg#L892
        - set stride=4 instead of https://github.com/AlexeyAB/darknet/blob/6f718c257815a984253346bba8fb7aa756c55090/cfg/yolov4.cfg#L989

    *   for training for both small and large objects use modified models:

        - Full-model: 5 yolo layers: https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3_5l.cfg
        - Tiny-model: 3 yolo layers: https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny_3l.cfg
        - YOLOv4: 3 yolo layers: https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-custom.cfg

    *   If you train the model to distinguish Left and Right objects as separate classes (left/right hand, left/right-turn on road signs, ...)     then for disabling flip data augmentation - add flip=0 here:                                                                               https://github.com/AlexeyAB/darknet/blob/3d2d0a7c98dbc8923d9ff705b81ff4f7940ea6ff/cfg/yolov3.cfg#L17

    *   General rule - your training dataset should include such a set of relative sizes of objects that you want to detect:

        - train_network_width * train_obj_width / train_image_width ~= detection_network_width * detection_obj_width / detection_image_width
        - train_network_height * train_obj_height / train_image_height ~= detection_network_height * detection_obj_height / detection_image_height
        I.e. for each object from Test dataset there must be at least 1 object in the Training dataset with the same class_id and about the same relative size:

            object width in percent from Training dataset ~= object width in percent from Test dataset

        That is, if only objects that occupied 80-90% of the image were present in the training set, then the trained network will not be able to detect objects that occupy 1-10% of the image.

    *   to speedup training (with decreasing detection accuracy) set param stopbackward=1 for layer-136 in cfg-file

    *   each: model of object, side, illimination, scale, each 30 grad of the turn and inclination angles - these are different objects from       an internal perspective of the neural network. So the more different objects you want to detect, the more complex network model           should be used.

    *   to make the detected bounded boxes more accurate, you can add 3 parameters ignore_thresh = .9 iou_normalizer=0.5 iou_loss=giou to each     [yolo] layer and train, it will increase mAP@0.9, but decrease mAP@0.5.

    *   Only if you are an expert in neural detection networks - recalculate anchors for your dataset for width and height from cfg-file:         darknet.exe detector calc_anchors data/obj.data -num_of_clusters 9 -width 416 -height 416 then set the same 9 anchors in each of 3         [yolo]-layers in your cfg-file. But you should change indexes of anchors masks= for each [yolo]-layer, so for YOLOv4 the 1st-[yolo]-       layer has anchors smaller than 30x30, 2nd smaller than 60x60, 3rd remaining, and vice versa for YOLOv3. Also you should change the         filters=(classes + 5)*<number of mask> before each [yolo]-layer. If many of the calculated anchors do not fit under the appropriate       layers - then just try using all the default anchors.

--> After training - for detection:
    *   Increase network-resolution by set in your .cfg-file (height=608 and width=608) or (height=832 and width=832) or (any value multiple       of 32) - this increases the precision and makes it possible to detect small objects: link

    *   it is not necessary to train the network again, just use .weights-file already trained for 416x416 resolution

    *   to get even greater accuracy you should train with higher resolution 608x608 or 832x832, note: if error Out of memory occurs then in       .cfg-file you should increase subdivisions=16, 32 or 64: link

                
## References:
    https://github.com/AlexeyAB/darknet
    https://github.com/AlexeyAB/darknet/wiki/Train-Detector-on-MS-COCO-(trainvalno5k-2014)-dataset

# Train YOLOv4 using tflite
https://github.com/hunglc007/tensorflow-yolov4-tflite


# PyTorch YOLOv4
    https://github.com/WongKinYiu/PyTorch_YOLOv4
    https://github.com/maudzung/Complex-YOLOv4-Pytorch
    https://github.com/Tianxiaomo/pytorch-YOLOv4#22-image-input-size-for-inference
    https://github.com/WongKinYiu/PyTorch_YOLOv4
    https://github.com/VCasecnikovs/Yet-Another-YOLOv4-Pytorch